## 4.3.2 数値微分の例

In [1]:
def function_1(x):
    return 0.01*x**2 + 0.1*x

In [2]:
import numpy as np
import matplotlib.pylab as plt

x = np.arange(0.0, 20.0, 0.1)
y = function_1(x)
plt.xlabel('x')
plt.ylabel('f(x)')
plt.plot(x, y)
plt.show()

<Figure size 640x480 with 1 Axes>

In [3]:
def numerical_diff(f, x):
    h = 1e-4
    return (f(x+h) - f(x-h)) / (2*h)

In [4]:
numerical_diff(function_1, 5)

0.1999999999990898

In [5]:
numerical_diff(function_1, 10)

0.2999999999986347

In [6]:
def function_2(x):
    return x[0]**2 + x[1]**2

In [7]:
def function_tmp1(x0):
    return x0*x0 + 4.0**2.0

In [8]:
numerical_diff(function_tmp1, 3.0)

6.00000000000378

In [9]:
def numerical_gradient(f, x):
    h = 1e-4
    grad = np.zeros_like(x)
    
    for idx in range(x.size):
        tmp_val = x[idx]
        # f(x+h)の計算
        x[idx] = tmp_val + h
        fxh1 = f(x)
        
        # f(x-h)の計算
        x[idx] = tmp_val - h
        fxh2 = f(x)
        
        grad[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val
        

    return grad

In [10]:
numerical_gradient(function_2, np.array([3.0,4.0]))

array([6., 8.])

In [11]:
numerical_gradient(function_2, np.array([0.0,2.0]))

array([0., 4.])

In [12]:
numerical_gradient(function_2, np.array([3.0,0.0]))

array([6., 0.])

In [13]:
def gradient_descent(f, init_x, lr = 0.01, step_num=100):
    x = init_x
    
    for i in range(step_num):
        grad = numerical_gradient(f, x)
        
        x -= lr * grad
    
    return x

In [14]:
def function_2(x):
    return x[0]**2 + x[1]**2

In [15]:
init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x=init_x, lr = 0.1, step_num=100)

array([-6.11110793e-10,  8.14814391e-10])

In [16]:
init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x=init_x, lr = 10.0, step_num=100)

array([-2.58983747e+13, -1.29524862e+12])

In [17]:
init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x=init_x, lr = 10e-10, step_num=100)

array([-2.9999994,  3.9999992])

## 4.4.2 ニューラルネットワークに対する勾配

In [18]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.functions import softmax, cross_entropy_error
from common.gradient import numerical_gradient

class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3)
        
    def predict(self, x):
        return np.dot(x, self.W)
    
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)
        
        return loss

In [19]:
net = simpleNet()
print(net.W)

[[ 1.32466528 -0.60007556 -0.60386757]
 [ 0.70541027 -0.61224961  0.61487819]]


In [20]:
x = np.array([0.6, 0.9])
p = net.predict(x)
print(p)

[ 1.42966841 -0.91106999  0.19106983]


In [21]:
np.argmax(p)

0

In [22]:
t = np.array([0, 0, 1])
net.loss(x, t)

1.5650536149292247

In [23]:
def f(W):
    return net.loss(x, t)

dW = numerical_gradient(f, net.W)
print(dW)

[[ 0.43288568  0.04166808 -0.47455375]
 [ 0.64932851  0.06250211 -0.71183063]]


In [24]:
f = lambda W: net.loss(x, t)
dW = numerical_gradient(f, net.W)
print(dW)

[[ 0.43288568  0.04166808 -0.47455375]
 [ 0.64932851  0.06250211 -0.71183063]]


## 4.5.1 2層ニューラルネットワークのクラス

In [25]:
import sys,os
sys.path.append(os.pardir)
from common.functions import *
from common.gradient import numerical_gradient

class TwoLayerNet:
    
    def __init__(self, input_size, hidden_size, output_size,
                weight_init_std=0.01):
        # 重みの初期化
        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']

        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)

        return y

    # x:入力データ, t:教師データ
    def loss(self, x, t):
        y = self.predict(x)

        return cross_entropy_error(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    # x:入力データ, t:教師データ
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}

        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

In [26]:
net = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)
print(net.params['W1'].shape)
print(net.params['b1'].shape)
print(net.params['W2'].shape)
print(net.params['b2'].shape)

(784, 100)
(100,)
(100, 10)
(10,)


In [27]:
x = np.random.rand(100, 784)
y = net.predict(x)

In [28]:
x = np.random.rand(100, 784)
t = np.random.rand(100, 10)

grads = net.numerical_gradient(x, t)

print(grads['W1'].shape)
print(grads['b1'].shape)
print(grads['W2'].shape)
print(grads['b2'].shape)

(784, 100)
(100,)
(100, 10)
(10,)


## 4.5.2 ミニバッチ学習の実装

In [ ]:
%%time
import numpy as np
from dataset.mnist import load_mnist
#from two_layer_net import TwoLayerNet

(x_train, t_train), (x_test, t_test) =\
    load_mnist(normalize=True, one_hot_label=True)

train_loss_list = []

# ハイパーパラメータ
iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iters_num):
    # ミニバッチの取得
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 勾配の計算
    grad = network.numerical_gradient(x_batch, t_batch)
    # grad = network.gradient(x_batch, t_batch) # 高速版!
    
    # パラメータの更新
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    # 学習経過の記録
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

In [ ]:
len(train_loss_list)